In [ ]:
!nvidia-smi

In [ ]:
!pip install transformers[sentencepiece] dataset sacrebleu rouge_score py7zr -q

In [ ]:
!pip install transformers==4.37.2  --user

In [ ]:
!pip install tf-keras==2.16 --user

In [ ]:
# To distribute the process between the cores inside GPU
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import AdamWeightDecay

### Using Open source language translation model from Hugging face to Language transaltion

In [ ]:
model = "Helsinki-NLP/opus-mt-en-hi"
dataset = load_dataset("cfilt/iitb-english-hindi")

In [ ]:
dataset

In [ ]:
 dataset["validation"]

In [ ]:
dataset["train"][10]

In [ ]:
# tokenizing the model
tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
tokenizer(["hello i am pramod", "pramod is my name"])

In [ ]:
tokenizer("हाइलाइट किया गया भराई का रंग और पारदर्शिता।")

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer("हाइलाइट किया गया भराई का रंग और पारदर्शिता।"))

In [ ]:
for ex in dataset["validation"]["translation"]:
    print(ex["hi"])
    print(ex["en"])

In [ ]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "hi"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    
    # Setting up the tokenizer for inputs
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_data = dataset.map(preprocess_function, batched=True)

In [ ]:
 tokenized_data

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model)

In [ ]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [ ]:
# for combining the dataset and tokens
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_data["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [ ]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_data["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [ ]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_data["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [ ]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

#### Inferencing

In [ ]:
model.save_pretrained("tf_model/")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

In [ ]:
input_text  = "I am learning Coding. How are you"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))